In [1]:
import numpy as np
from scipy import signal as sg
from time import time

In [2]:
def formHi(v,N1):
    R=np.array([v]).T
    
    for i in range(N1-1):
        v=np.insert(v,0,v[-1])
        v=np.delete(v,-1)
        R=np.hstack((R,np.array([v]).T))
    return R

def formH(Hlist,M1,N3):
    
    vertical=Hlist[0]
    for i in range(1,len(Hlist)):
        vertical=np.vstack((vertical,Hlist[i]))
    
    R=vertical.copy()
    
    for _ in range(M1-1):
        for i in range(N3):
            vertical=np.insert(vertical,0,vertical[-1-i],axis=0)
        for i in range(N3):
            vertical=np.delete(vertical,-1,axis=0)
        R=np.hstack((R,vertical))
    
    return R

def formF(A):
    R=np.array([])
    for row in A:
        R=np.append(R,row)
    return R.reshape(-1,1)

def toeplitzConvolution(A,K):
    M1,N1=A.shape
    M2,N2=K.shape
    M3=M1+M2-1
    N3=N1+N2-1
    
    K_padded=np.zeros((M3,N3))
    K_padded[:M2,:N2]=K
    
    f=formF(A)
    
    Hlist=[]
    for row in K_padded:
        Hlist.append(formHi(row,N1))
    
    H=formH(Hlist,M1,N3)
    
    return np.dot(H,f).reshape(M3,N3)

def circularConvolution(v,k):
    C=np.array([])
    C=v.reshape(-1,1)
    for i in range(v.size-k.size):
        k=np.append(k,0)
        
    for i in range(v.size-1):
        v=np.insert(v,0,v[-1])
        v=np.delete(v,-1)
        C=np.hstack((C,np.array([v]).T))
    return np.dot(C,k),C

For periodic input x=[1 8 3 2 5] and system response h=[3 5 2 4 1]:

In [16]:
x,C=circularConvolution(np.array([1,8,3,2,5]),np.array([3,5,2,4,1]))
print('x*h =',x)
print('C = \n',C)

x*h = [52 50 73 46 64]
C = 
 [[1 5 2 3 8]
 [8 1 5 2 3]
 [3 8 1 5 2]
 [2 3 8 1 5]
 [5 2 3 8 1]]


For every Circulant matrix C, one eigen vector is always [1,1,...,1] and corresponding eigen value is sum of all elements in one row of matrix C

In [18]:
print('Eigen vector:',np.ones(C.shape[0]))
print('Corresponding eigen value:',np.sum(C[0]))

Eigen vector: [1. 1. 1. 1. 1.]
Corresponding eigen value: 19


For Circulant matrices C1 and C2 of the same shape,all eigen vectors are the same:

In [53]:
x1,C1=circularConvolution(np.random.randint(0,100,10),np.array([1,2,3]))
x2,C2=circularConvolution(np.random.randint(0,100,10),np.array([1,2,3]))

vectors1=np.linalg.eig(C1)[1]
vectors2=np.linalg.eig(C2)[1]
eig_vectors_diff=[]
for i in range(vectors1.shape[1]):
    eig_vectors_diff.append(sum(vectors1[:,i]-vectors2[:,i]))
    
print(np.sum(eig_vectors_diff))

(-1.3600232051658168e-15-1.3877787807814457e-16j)


In [34]:
a=np.random.randint(0,100,(50,50))
b=np.random.randint(0,10,(3,3))
t0=time()
y=sg.convolve2d(a,b)
t1=time()
x=toeplitzConvolution(a,b)
t2=time()

print(sum(x-y))
print('signal.convolve2d:',t1-t0)
print('toeplitz convolution:',-t1+t2)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
signal.convolve2d: 0.0004973411560058594
toeplitz convolution: 1.1964592933654785
